### Setting up graphviz used to convert Decision Trees into graphs, which are then rendered.

#### (Switch from markdown to code if you would like to run.)

import sys

!{sys.executable} -m pip install graphviz

#### Note: graphviz needs to be installed through ``brew`` or ``apt-get``, for instance, as this is only a python binding.

### Interactively fitting a Decision Tree.

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor, export_graphviz

from graphviz import Source

from IPython.display import SVG
from IPython.display import display                               
from ipywidgets import interactive

#### Learning opportunities: discuss the original data, discuss feature selection, reduce feature space down to ~10

In [12]:
df_crimes = pd.read_csv('communities.data')
df_crimes.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [15]:
list(df_crimes.columns) # Refer to communities.properties for feature description

['state',
 'county',
 'community',
 'communityname',
 'fold',
 'population',
 'householdsize',
 'racepctblack',
 'racePctWhite',
 'racePctAsian',
 'racePctHisp',
 'agePct12t21',
 'agePct12t29',
 'agePct16t24',
 'agePct65up',
 'numbUrban',
 'pctUrban',
 'medIncome',
 'pctWWage',
 'pctWFarmSelf',
 'pctWInvInc',
 'pctWSocSec',
 'pctWPubAsst',
 'pctWRetire',
 'medFamInc',
 'perCapInc',
 'whitePerCap',
 'blackPerCap',
 'indianPerCap',
 'AsianPerCap',
 'OtherPerCap',
 'HispPerCap',
 'NumUnderPov',
 'PctPopUnderPov',
 'PctLess9thGrade',
 'PctNotHSGrad',
 'PctBSorMore',
 'PctUnemployed',
 'PctEmploy',
 'PctEmplManu',
 'PctEmplProfServ',
 'PctOccupManu',
 'PctOccupMgmtProf',
 'MalePctDivorce',
 'MalePctNevMarr',
 'FemalePctDiv',
 'TotalPctDiv',
 'PersPerFam',
 'PctFam2Par',
 'PctKids2Par',
 'PctYoungKids2Par',
 'PctTeen2Par',
 'PctWorkMomYoungKids',
 'PctWorkMom',
 'NumIlleg',
 'PctIlleg',
 'NumImmig',
 'PctImmigRecent',
 'PctImmigRec5',
 'PctImmigRec8',
 'PctImmigRec10',
 'PctRecentImmig',
 'P

#### Preprocessing so it takes in numerical features to predict a numerical target

In [16]:
df_crimes.drop('communityname', inplace=True, axis=1) # Drop categorical data
df_cleaned = df_crimes.replace({'?': np.nan}).dropna() # Drop missing values

In [17]:
len(df_cleaned)

123

#### Remove/add features in order to discuss bias vs. accuracy tradeoff
#### TO-DOs:
#### 1. Measure accuracy in the first place
#### 2. Create a widget to turn on/off features

#### "If you send police to a neighborhood based on race/ethnicity then you're injecting bias in crime measurement data, which would cause the ML model to be (explicitly) racially discriminatory"

In [5]:
ratio = 1.0
pct_black_median = None


# If a given row x (a training example) has 'racepctblack' greater than the median value, then
# multiply the outcome 'ViolentCrimesPerPop' by the user-defined ratio; otherwise, multiply by
# the inverse of the user-defined ratio.
# Interpretation: the greater the ratio, the worst is the bias against communities with above
# average Black populations; the smaller the ratio, the more favorable is the bias; setting to
# 1.0 keeps the data in its original form.

def apply_bias(x):
    global ratio, pct_black_median
    return x['ViolentCrimesPerPop']*ratio if x['racepctblack'] > pct_black_median else x['ViolentCrimesPerPop']*(1/ratio)


def plot_tree(overpolicing=1.0):
    global ratio, pct_black_median # Made global to interface with function apply_bias
    ratio = overpolicing # Set the amount of bias to whatever the user defines through the slider
    
    # Load the data from scratch every time
    df_crimes = pd.read_csv('communities.data')
    df_crimes.drop('communityname', inplace=True, axis=1) # Drop categorical data
    df_cleaned = df_crimes.replace({'?': np.nan}).dropna() # Drop missing values
    
    # Median also used in function apply_bias
    pct_black_median = df_cleaned['racepctblack'].median()
    
    df_biased = df_cleaned
    df_biased['ViolentCrimesPerPop'] = df_biased.apply(apply_bias, axis=1)
    
    # Vanilla ML
    X = df_biased[df_biased.columns[:-1]]
    Y = df_biased['ViolentCrimesPerPop']
    estimator = DecisionTreeRegressor(random_state=0, criterion='mse', splitter= 'best', max_depth=5)
    estimator.fit(X, Y)
    
    # Render Decision Tree
    labels = X.columns
    graph = Source(export_graphviz(estimator, out_file=None, feature_names=labels, filled = True))
    display(SVG(graph.pipe(format='svg')))
    return estimator


inter=interactive(plot_tree, bias=1.0)
display(inter)

interactive(children=(FloatSlider(value=1.0, description='overpolicing', max=3.0, min=-1.0), Output()), _dom_c…